author: me bitch

## Notebook to test various NLTK functions. ## 

General procedure for test data:
- Break words into tokens
- Strip tokens of punctuation
- Remove commonly used words that do not provide meaningful content

Test data with be tested against trained data. I'll be using data from the *NLTK corpus* to train the classifier. Testing **Naive Bayes Classifier** first. 

Currently looking into sentiment analysis (check bottom) to test whether a tweet is positive or negative.
The project spec gives an example of topic modeling. Which utilises **LDA** to classify the features (topics) of tweets. You can then generate, for instance, a list of top ten tweet topics or how certain topics are related. 

The latter example is fairly simple compared to the former one. So will probably try to implement a version of the latter as soon as possible.

Most of the classifier stuff we did in machine learning, I just need to figure out the structure of the data (tweets) and with what trained data am I going to compare with.

Also, I'm using Jupyter notebook for quick testing purposes. 

**tl;dr**
* Get the topic of a tweet and how it relates to other topics.
* Sentiment analysis: Is a tweet negative or positive in nature.


<div class="alert alert-block alert-info">
<b>Disclaimer:</b>You'll probably need to download a bunch of data libraries to run this notebook. Not needed at this moment. This is just testing data. I'm just uploading this to show my work in progress and to get some feedback. 
</div>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from string import punctuation
import random

import nltk

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import ne_chunk, pos_tag

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.corpus import names
from nltk.tokenize import PunktSentenceTokenizer
from nltk.corpus import movie_reviews

import pickle

# will probably use these later for plotting
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import style
import time

In [4]:
text = "Tabs that can be selected to see all your tweets, with number of retweets (by username of person that did the retweet) and likes indicated, ordered from most recent to oldest, a list of people you	are	following, and a list of people who are following you."

# splits the text into token words
stop_words = stopwords.words('english')
words = word_tokenize(text)

# removes punctuation an dcommon words that don't contribute towards the text
without_punctuation = []
without_stop_words = []

for w in words:
    if w not in punctuation:
        without_punctuation.append(w)
        
for w in without_punctuation:
    if w not in stop_words:
        without_stop_words.append(w)        

print(without_stop_words)

# reduces words to its stem (root word)
ps = PorterStemmer()
for w in without_stop_words:
    print(ps.stem(w))

['Tabs', 'selected', 'see', 'tweets', 'number', 'retweets', 'username', 'person', 'retweet', 'likes', 'indicated', 'ordered', 'recent', 'oldest', 'list', 'people', 'following', 'list', 'people', 'following']
tab
select
see
tweet
number
retweet
usernam
person
retweet
like
indic
order
recent
oldest
list
peopl
follow
list
peopl
follow


Testing getting parts of speech of each token. Not important

In [ ]:
# labels each word with its parts of speech
train_text = names.raw("female.txt")
sample_text = names.raw("male.txt")

custom_sent_tokenizer = PunktSentenceTokenizer(train_text)
tokenized = custom_sent_tokenizer.tokenize(sample_text)

def process_content():
    try:
        for i in tokenized:
            words = nltk.word_tokenize(i)
            tagged = nltk.pos_tag(words)
            print(tagged)
            
    except Exception as e:
        print(str(e))
        
process_content()

Below is a rough implementation of sentiment analysis using **Naive Bayes Classifier**.  Still early phase. Using movie reviews data from NLTK as training set.

In [6]:
documents = [(list(movie_reviews.words(fileid)), category) 
             for category in movie_reviews.categories()
            for fileid in movie_reviews.fileids(category)]

random.shuffle(documents)

all_words = []
for w in movie_reviews.words():
    all_words.append(w.lower())
    
all_words = nltk.FreqDist(all_words)

word_features = list(all_words.keys())[:3000]

def find_features(document):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
        
    return features

featuresets = [(find_features(rev), category) for (rev, category) in documents]

training_set = featuresets[:1900]
testing_set = featuresets[1900:]

classifier = nltk.NaiveBayesClassifier.train(training_set)
print("Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(15)


('Naive Bayes Algo accuracy percent:', 72.0)
Most Informative Features
               insulting = True              neg : pos    =     10.6 : 1.0
              unoriginal = True              neg : pos    =      9.6 : 1.0
                    sans = True              neg : pos    =      9.0 : 1.0
            refreshingly = True              pos : neg    =      7.7 : 1.0
              mediocrity = True              neg : pos    =      7.6 : 1.0
                   tripe = True              neg : pos    =      7.6 : 1.0
               dismissed = True              pos : neg    =      7.0 : 1.0
             bruckheimer = True              neg : pos    =      6.3 : 1.0
                  doubts = True              pos : neg    =      5.8 : 1.0
                  wasted = True              neg : pos    =      5.8 : 1.0
                  stinks = True              neg : pos    =      5.8 : 1.0
                    wits = True              pos : neg    =      5.7 : 1.0
                    lang = Tr